In [ ]:
import copy
from types import SimpleNamespace

import vrplib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
import math

from alns import ALNS
from alns.accept import *
from alns.select import *
from alns.stop import *

In [ ]:
%matplotlib inline

In [ ]:
SEED = 1234

In [ ]:
"""
Read Source Data
"""
data = vrplib.read_instance('data/ORTEC-n242-k12.vrp')
bks = SimpleNamespace(**vrplib.read_solution('data/ORTEC-n242-k12.sol'))

In [ ]:
def plot_solution(solution, name="CVRP solution"):
    """
    Plot the routes of the passed-in solution.
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    cmap = matplotlib.cm.rainbow(np.linspace(0, 1, len(solution.routes)))

    for idx, route in enumerate(solution.routes):
        ax.plot(
            [data["node_coord"][loc][0] for loc in [0] + route + [0]],
            [data["node_coord"][loc][1] for loc in [0] + route + [0]],
            color=cmap[idx],
            marker='.'
        )

    # Plot the depot
    kwargs = dict(label="Depot", zorder=3, marker="*", s=750)
    ax.scatter(*data["node_coord"][0], c="tab:red", **kwargs)

    ax.set_title(f"{name}\n Total distance: {solution.cost}")
    ax.set_xlabel("X-coordinate")
    ax.set_ylabel("Y-coordinate")
    ax.legend(frameon=False, ncol=3)

In [ ]:
class CvrpState:
    """
    Solution state for CVRP. It has two data members, routes and unassigned.
    Routes is a list of list of integers, where each inner list corresponds to
    a single route denoting the sequence of customers to be visited. A route
    does not contain the start and end depot. Unassigned is a list of integers,
    each integer representing an unassigned customer.
    """

    def __init__(self, routes, unassigned=None):
        self.routes = routes
        self.unassigned = unassigned if unassigned is not None else []

    def copy(self):
        return CvrpState(copy.deepcopy(self.routes), self.unassigned.copy())

    def objective(self):
        """
        Computes the total route costs.
        """
        return sum(route_cost(route) for route in self.routes)

    @property
    def cost(self):
        """
        Alias for objective method. Used for plotting.
        """
        return self.objective()

    def find_route(self, customer):
        """
        Return the route that contains the passed-in customer.
        """
        for route in self.routes:
            if customer in route:
                return route

        raise ValueError(f"Solution does not contain customer {customer}.")

def route_cost(route):
    distances = data["edge_weight"]
    a = 8.5e-7
    b = 0.002
    batteryHealth = a*distances+b
    tour = [0] + route + [0]

    return sum(batteryHealth[tour[idx]][tour[idx + 1]]
               for idx in range(len(tour) - 1))

In [ ]:
"""
Funtions to generate the initial solution.
"""

def neighbors(customer):
    """
    Return the nearest neighbors of the customer, excluding the depot.
    """
    locations = np.argsort(data["edge_weight"][customer])
    return locations[locations != 0]

def nearest_neighbor():
    """
    Build a solution by iteratively constructing routes, where the nearest
    customer is added until the route has met the vehicle capacity limit.
    """
    routes = []
    unvisited = set(range(1, data["dimension"]))

    while unvisited:
        route = [0]  # Start at the depot
        route_demands = 0

        while unvisited:
            # Add the nearest unvisited customer to the route till max capacity
            current = route[-1]
            nearest = [nb for nb in neighbors(current) if nb in unvisited][0]

            if route_demands + data["demand"][nearest] > data["capacity"]:
                break

            route.append(nearest)
            unvisited.remove(nearest)
            route_demands += data["demand"][nearest]

        customers = route[1:]  # Remove the depot
        routes.append(customers)

    return CvrpState(routes)

In [ ]:
"""
Random Removal Heuristic
"""

degree_of_destruction = 0.05
customers_to_remove = int((data["dimension"] - 1) * degree_of_destruction)

def random_removal(state, rnd_state):
    """
    Removes a number of randomly selected customers from the passed-in solution.
    """
    destroyed = state.copy()

    for customer in rnd_state.choice(
        range(1, data["dimension"]), customers_to_remove, replace=False
    ):
        destroyed.unassigned.append(customer)
        route = destroyed.find_route(customer)
        route.remove(customer)

    return remove_empty_routes(destroyed)


def remove_empty_routes(state):
    """
    Remove empty routes after applying the destroy operator.
    """
    state.routes = [route for route in state.routes if len(route) != 0]
    return state


In [ ]:
"""
Worst Removal Heuristic
"""

def worst_removal(state, rnd_state):
    """
    Removes customers that bring the most cost to the route.
    """
    destroyed = state.copy()

    newCost=[]
    customers=[]
    # Sort route by cost difference, worst first.
    for route in range(len(destroyed.routes)):
        for idx in range(len(destroyed.routes[route])):
            customers.append(destroyed.routes[route][idx])
            newCost.append(cost_difference(destroyed,destroyed.routes[route][idx]))

    sortedNewCost = sorted(enumerate(newCost),key= lambda newCost:newCost[1],reverse=True)
    newIndex = [sortedNewCost[0] for sortedNewCost in sortedNewCost]

    # Pick a random customer by the randomParameter.
    for idx in range(customers_to_remove):
        y = np.random.random_sample()
        pWorst = 3
        randomParameter = math.floor(y**pWorst)*len(newIndex)
        r = customers[newIndex[randomParameter]]
        destroyed.unassigned.append(r)
        route = destroyed.find_route(r)
        route.remove(r)
        newIndex.remove(newIndex[randomParameter])
             
    return remove_empty_routes(destroyed)

def cost_difference(state,customer):
    """
    Compute the cost difference if this customer is removed from the route.
    """
    newRoute = state.copy()
    newRoute.unassigned.append(customer)
    route = newRoute.find_route(customer)
    route.remove(customer)
    cost = route_cost(route)
    return cost

In [ ]:
"""
Long Distance Removal Heuristic
"""

def short_removal(state, rnd_state):
    """
    Removes a number of selected customers that is the front point of a loong distance edge.
    """
    destroyed = state.copy()
    shape = tuple(len(row) for row in destroyed.routes)
    longestRouteIndex = shape.index(max(shape)) # Find the route that has the most customers.
    chosenRoute = destroyed.routes[longestRouteIndex]
    rnd_Customer = np.random.randint(0,max(shape)-1)

    bar_distance = data["edge_weight"][chosenRoute[rnd_Customer]][chosenRoute[rnd_Customer+1]] # Compute the criterion of the  long distance for this iteration.
            
    chosenCustomers = []
    for route in range(len(destroyed.routes)):
        for idx in range(len(destroyed.routes[route])-1):
            distance = data["edge_weight"][destroyed.routes[route][idx]][destroyed.routes[route][idx+1]]
            if distance > bar_distance:
                chosenCustomers.append(destroyed.routes[route][idx])

    for customer in chosenCustomers:
        destroyed.unassigned.append(customer)
        route = destroyed.find_route(customer)
        route.remove(customer)

    return remove_empty_routes(destroyed)

In [ ]:
"""
Slack Removal Heuristic
"""

MAX_STRING_REMOVALS = 2
MAX_STRING_SIZE = 12

def string_removal(state, rnd_state):
    """
    Remove partial routes around a randomly chosen customer.
    """
    destroyed = state.copy()

    avg_route_size = int(np.mean([len(route) for route in state.routes]))
    max_string_size = max(MAX_STRING_SIZE, avg_route_size)
    max_string_removals = min(len(state.routes), MAX_STRING_REMOVALS)

    destroyed_routes = []
    center = rnd_state.randint(1, data["dimension"])

    for customer in neighbors(center):
        if len(destroyed_routes) >= max_string_removals:
            break

        if customer in destroyed.unassigned:
            continue

        route = destroyed.find_route(customer)
        if route in destroyed_routes:
            continue

        customers = remove_string(route, customer, max_string_size, rnd_state)
        destroyed.unassigned.extend(customers)
        destroyed_routes.append(route)

    return destroyed


def remove_string(route, cust, max_string_size, rnd_state):
    """
    Remove a string that constains the passed-in customer.
    """
    # Find consecutive indices to remove that contain the customer
    size = rnd_state.randint(1, min(len(route), max_string_size) + 1)
    start = route.index(cust) - rnd_state.randint(size)
    idcs = [idx % len(route) for idx in range(start, start + size)]

    # Remove indices in descending order
    removed_customers = []
    for idx in sorted(idcs, reverse=True):
        removed_customers.append(route.pop(idx))

    return removed_customers


In [ ]:
"""
Greedy Repair Heuristic
"""

def greedy_repair(state, rnd_state):
    """
    Inserts the unassigned customers in the best route. If there are no
    feasible insertions, then a new route is created.
    """
    rnd_state.shuffle(state.unassigned)

    while len(state.unassigned) != 0:
        customer = state.unassigned.pop()
        cost, route, idx = best_insert(customer, state)

        if route is not None:
            route.insert(idx, customer)
        else:
            state.routes.append([customer])

    return state

def best_insert(customer, state):
    """
    Finds the best feasible route and insertion idx for the customer.
    Return (None, None) if no feasible route insertions are found.
    """
    best_cost, best_route, best_idx = None, None, None
    for route in state.routes:
        for idx in range(len(route) + 1):

            if can_insert(customer, route):
                cost = insert_cost(customer, route, idx)

                if best_cost is None or cost < best_cost:
                    best_cost, best_route, best_idx = cost, route, idx
    return best_cost, best_route, best_idx


def can_insert(customer, route):
    """
    Checks if inserting customer does not exceed vehicle capacity.
    """
    total = data["demand"][route].sum() + data["demand"][customer]
    return total <= data["capacity"]


def insert_cost(customer, route, idx):
    """
    Computes the insertion cost for inserting customer in route at idx.
    """
    #a = 8.5e-7
    #b = 0.002
    #dist = a*data["edge_weight"]+b
    dist = data["edge_weight"]
    pred = 0 if idx == 0 else route[idx - 1]
    succ = 0 if idx == len(route) else route[idx]

    # Increase in cost of adding customer, minus cost of removing old edge
    return dist[pred][customer] + dist[customer][succ] - dist[pred][succ]

In [ ]:
"""
Regret K Heuristic
"""

def regret_k_repair(state, rnd_state):
    """
    Compute the regret values of unassigned customers and find the best route for them by the order of regret values.
    """

    rnd_state.shuffle(state.unassigned)
    k = 1 # The number of customers will be processed at once.
    
    while len(state.unassigned) != 0:
        regretSet = []
        for i in range(len(state.unassigned)):
            customer = state.unassigned[i]
            regret = regret_value(customer,state)
            regretSet.append(regret)
    
        regretSetSorted = sorted(enumerate(regretSet), key=lambda regretSet:regretSet[1], reverse=True) # The order of regret values.
        customerIdx = [x[0] for x in regretSetSorted]
        
        if len(state.unassigned)>=k:
            customers = [state.unassigned[customerIdx[i]] for i in range(k)]
        else:
            customers = state.unassigned

        for customer in customers:

            state.unassigned.remove(customer)
            cost, route, idx = best_insert(customer, state)

            if route is not None:
                route.insert(idx, customer)
            else:
                state.routes.append([customer])    

    return state

def regret_value(customer, state):
    """
    Compute the regret values (the difference between the best feasible route and the second best feasible route).
    """
    best_cost, best_route, best_idx = best_insert(customer, state)
    second_cost, second_route, second_idx = second_insert(customer, state, best_cost, best_route, best_idx)

    if best_cost is None or second_cost is None:
        regret = 0
    else:
        regret = second_cost - best_cost

    return regret

def second_insert(customer, state, best_cost, best_route, best_idx):
    """
    Finds the second best feasible route and insertion idx for the customer.
    Return (None, None) if no feasible route insertions are found.
    """
    second_cost, second_route, second_idx = None, None, None

    for route in state.routes:
        for idx in range(len(route) + 1):

            if can_insert(customer, route):
                cost = insert_cost(customer, route, idx)
                if cost != best_cost:
                    if second_cost is None or cost < second_cost:
                        second_cost, second_route, second_idx = cost, route, idx
                elif  not (route == best_route and idx == best_idx):
                    second_cost, second_route, second_idx = cost, route, idx
                        
    
    return second_cost, second_route, second_idx


In [ ]:
plot_solution(nearest_neighbor(), 'Nearest neighbor solution')

In [ ]:
alns = ALNS(rnd.RandomState(SEED))

#alns.add_destroy_operator(worst_removal)
alns.add_destroy_operator(short_removal)
alns.add_destroy_operator(string_removal)
alns.add_destroy_operator(random_removal)

alns.add_repair_operator(greedy_repair)
#alns.add_repair_operator(regret_k_repair)

In [ ]:
init = nearest_neighbor()
select = RouletteWheel([25, 5, 1, 0], 0.8, 3, 1)
#select = SegmentedRouletteWheel([25, 5, 1, 0], 0.8, 500, 3, 1)
#select = AlphaUCB(scores=[25, 5, 1, 0], alpha=0.05, num_destroy=3, num_repair=1)
accept = RecordToRecordTravel.autofit(init.objective(), 0.02, 0, 150000)
stop = MaxRuntime(3600)

result = alns.iterate(init, select, accept, stop)

In [ ]:
solution = result.best_state
objective = solution.objective()

bestSolution = CvrpState(bks.routes)
pct_diff = 100 * (objective - bestSolution.cost) / bestSolution.cost
print(f"Best heuristic objective is {objective}.")
print(f"This is {pct_diff:.1f}% worse than the optimal solution, which is {bestSolution.cost}.")

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))
result.plot_objectives(ax=ax)

In [ ]:
plot_solution(solution, 'My ALNS')